## Request NEXRAD Level-2 Radar Data.
This notebook is an example of how to use pydrology to download NEXRAD level-2 doppler radar data. NOAA provides access to NEXRAD doppler data across the contiguous United States. More information can be found here (https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ncdc:C00345). With this script, you can requeset raw NEXRAD Level-2 Base data and then further process it into a NetCDF4 file if desired. 

The general workflow for requesting data is as follows:
1. Choose a doppler radar site calling code from this map (https://www.roc.noaa.gov/WSR88D/Maps.aspx).
2. Download the data with a request to an output directory.
3. Process the reflectivity parameter into a NetCDF4 file if desired.

In [ ]:
from pydrology import nexrad_download, nexrad_processing
from pathlib import Path
import netCDF4 as nc4
import subprocess

## 1 & 2) Request/Download NEXRAD files.

In [ ]:
# Parameters for data request.
site = 'KBGM'
date = '2022-07-25'
download_dir = Path.home() / 'Desktop' / 'NEXRAD_Output'
threads = 10

### Option 1: Download with function call through Jupyter Notebook
The following cell shows how to download a single day of NEXRAD scans with a function call in the Jupyter
Notebook.

** Note that logging shows up as red and is not an error or warning if it does not say explicitly.

In [ ]:
# Request and download NEXRAD data. Output is a list of file paths to downloaded NEXRAD files.
nexrad_download.main(download_dir, date, site, threads)

### Option 2: Download with a command line call
You can directly call the script nexrad_download.py with a call in the command line. The arguments are as follows:

```
>> python3 nexrad_download.py -h

Download NEXRAD Scans from a single day.

optional arguments:
  -h, --help         show this help message and exit
  --dir DIR          Output directory. Files will be saved in sub-directory using the
                     date.
  --threads THREADS  Number of download threads to use at once.
  --date DATE        Date of the scans yyyy-mm-dd
  --site SITE        NEXRAD site code.
```

A sample call might look like the following.

```
>> python3 nexrad_download.py --dir ~/NEXRAD_Output --threads 10 --date 2022-07-18 --site KBGM
```

In [ ]:
# With iPython shell.
nexrad_download_fp = Path.cwd().parents[0] / 'pydrology' / 'nexrad_download.py'

command = f'python3 {nexrad_download_fp} --dir {download_dir} --threads {threads} --date {date} --site {site}'
!{command}

## 3) Create NetCDF file from output data. 

This step must be done using a shell or from the command line. Creating a netCDF file from the data utilizes mulitprocessing, but due to Python's GIL, it's not possible to call the script normally from a Jupyter Notebook. Nexrad processing also includes several required parameters and some optional ones for extra processing. The example configuration file can be found under examples/config_files/nexrad_config.yml.


#### Example call using the --config argument.

```
>> python3 nexrad_processing.py -h

Process NEXRAD Data from a single day.

optional arguments:
  -h, --help       show this help message and exit
  --config CONFIG  Path to NEXRAD processing configuration file.

```

A call to process NEXRAD scans into a NetCDF file might look like the following:

```
>> python3 nexrad_processing.py --config ~/nexrad_config.yml
```

In [ ]:
# Parameters for processing.

# Path to nexrad_processing.py
nexrad_processing_fp = Path.cwd().parents[0] / 'pydrology' / 'nexrad' / 'nexrad_processing.py'

# Path to nexrad_config.yml.
nexrad_config_fp = Path.cwd().parents[0] / 'examples' / 'config_files' / 'nexrad_config.yml'

#### Option 1: Run with iPython shell

In [ ]:
# Run with iPython shell.
command = f'python3 {nexrad_processing_fp} --config {nexrad_config_fp}'
!{command}

#### Option 2: Run with subprocess

In [ ]:
# Populate command.
command = [
    'python3',
    nexrad_processing_fp,
    '--dir', download_dir,
    '--pool', str(Npool),
    '--date', date,
    '--site', site,
    '--var', var
]

# Run with subprocess.
p = subprocess.call(args)

### Example NetCDF file.
Below is what the contents of a NetCDF file of NEXRAD scans for a single day look like. This file was created from the downloaded scans on 2022-07-25 from station KBGM. 

In [ ]:
# Create netCDF4 file from output files.
nc_path = Path.home() / 'Desktop' / 'NEXRAD_Output' / 'NEXRAD_KBGM_2022-07-25.nc'

# Inspect the netCDF4 file.
rootgrp = nc4.Dataset(nc_path, "r", format="NETCDF4")

print('\n', 'NETCDF INSPECTION')
print('\n', 'DIMENSIONS')
for key, val in rootgrp.dimensions.items():
    print(key, ':', val)
    
print('\n', 'VARIABLES')
for key, val in rootgrp.variables.items():
    print(key, ':', val)

rootgrp.close()